In [ ]:
!pip install datasets evaluate scikit-learn transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.5 MB/s eta 0:00:00


In [ ]:
!pip install evaluate

Loading Libraries

In [ ]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 58.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
!pip install transformers[torch]

  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)


In [ ]:
%%capture

!pip install datasets evaluate scikit-learn transformers

from IPython.display import display, HTML
from google.colab import files

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import sklearn.metrics
import evaluate
import pandas as pd
import numpy as np
import json
import os
from io import StringIO
import time
from tqdm import tqdm


from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, TrainingArguments, Trainer, EarlyStoppingCallback, set_seed

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Session Info

In [ ]:
session_info = json.loads(os.popen("curl curl ipinfo.io").read())
if device=='cuda':
  gpu_info = pd.read_csv(StringIO(os.popen("nvidia-smi --query-gpu=gpu_name,memory.total --format=csv").read()),names=["name","memory"],header=0)
  session_info[f'gpus'] = [{'name': row["name"], 'memory': row["memory"]} for index, row in gpu_info.iterrows()]
else:
  session_info[f'gpus'] = []
session_info['time_start'] = time.strftime("%H:%M:%S", time.localtime())
session_info

{'ip': '34.125.208.221',
 'hostname': '221.208.125.34.bc.googleusercontent.com',
 'city': 'Las Vegas',
 'region': 'Nevada',
 'country': 'US',
 'loc': '36.1750,-115.1372',
 'org': 'AS396982 Google LLC',
 'postal': '89111',
 'timezone': 'America/Los_Angeles',
 'readme': 'https://ipinfo.io/missingauth',
 'gpus': [{'name': 'Tesla T4', 'memory': ' 15360 MiB'}],
 'time_start': '03:23:09'}

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

with open('/content/drive/MyDrive/Train.PubTator', 'r') as f:
    lines = f.readlines()

entity_file = open('train_ANN.txt', 'w')
corpus_file = open('train_CORPUS.json', 'w')
relation_file = open('train_REL.txt', 'w')

corpus_data_list = []

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 6:
       entity_file.write('\t'.join(parts[0:6]) + '\n')

    if len(parts) == 1 and '|a|' in parts[0]:
       parts_0 = parts[0].split('|a|')

       PMID = parts_0[0]
       sentence = parts_0[1]

       corpus_data = {
         "PMID": PMID,
         "Testo": sentence
         }

       corpus_data_list.append(corpus_data)


    if len(parts) == 5:
        relation_file.write('\t'.join(parts[:5]) + '\n')

corpus_file.write(json.dumps({"data": corpus_data_list}))

entity_file.close()
corpus_file.close()
relation_file.close()

In [ ]:
import json

with open('/content/drive/MyDrive/Test.PubTator', 'r') as f:
    lines = f.readlines()

entity_file = open('test_ANN.txt', 'w')
corpus_file = open('test_CORPUS.json', 'w')
relation_file = open('test_REL.txt', 'w')

corpus_data_list = []

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 6:
       entity_file.write('\t'.join(parts[0:6]) + '\n')

    if len(parts) == 1 and '|a|' in parts[0]:
       parts_0 = parts[0].split('|a|')

       PMID = parts_0[0]
       sentence = parts_0[1]

       corpus_data = {
         "PMID": PMID,
         "Testo": sentence
         }

       corpus_data_list.append(corpus_data)


    if len(parts) == 5:
        relation_file.write('\t'.join(parts[:5]) + '\n')

corpus_file.write(json.dumps({"data": corpus_data_list}))

entity_file.close()
corpus_file.close()
relation_file.close()

In [ ]:
import json

with open('/content/drive/MyDrive/Dev.PubTator', 'r') as f:
    lines = f.readlines()

entity_file = open('dev_ANN.txt', 'w')
corpus_file = open('dev_CORPUS.json', 'w')
relation_file = open('dev_REL.txt', 'w')

corpus_data_list = []

for line in lines:
    parts = line.strip().split('\t')
    if len(parts) == 6:
       entity_file.write('\t'.join(parts[0:6]) + '\n')

    if len(parts) == 1 and '|a|' in parts[0]:
       parts_0 = parts[0].split('|a|')

       PMID = parts_0[0]
       sentence = parts_0[1]

       corpus_data = {
         "PMID": PMID,
         "Testo": sentence
         }

       corpus_data_list.append(corpus_data)


    if len(parts) == 5:
        relation_file.write('\t'.join(parts[:5]) + '\n')

corpus_file.write(json.dumps({"data": corpus_data_list}))

entity_file.close()
corpus_file.close()
relation_file.close()

**Data Format Example**

{'guid': 'document_00000',
 'sentence': 'We therefore tested the hypothesis that variability in the HNF-6 gene is associated with subsets of Type II (non-insulin-dependent) diabetes mellitus and estimates of insulin secretion in glucose tolerant subjects.',

 'subject_entity': {'word': 'HNF-6',
                    'start_idx': 45,
                    'end_idx': 54,
                    'type': 'GeneOrGeneProduct'},

 'object_entity': {'word': 'type II diabetes',
                   'start_idx': 90,
                   'end_idx': 115,
                   'type': 'DiseaseOrPhenotypicFeature'},
  'label': 1,
  'source': 'wikipedia'}

Related Labels Example

{0:"No Relation",
 1:"Association",
 2:"Positive Correlation",
 3:"Negative Correlation"}

In [ ]:
def recompute_indices(sentence, match, start, end, window = 20):
  idx=-1
  while idx==-1:
    newstart = start-window if (start-window)>=0 else 0
    newend = end+window if (end+window)<len(sentence) else len(sentence)
    substring = sentence[newstart:newend]
    idx = substring.find(match)
    window = window+window
    if newstart==0 and newend==len(sentence) and idx==-1:
      return -1,-1
  return newstart+idx,newstart+idx+len(match)-1

In [ ]:
import pdb
import warnings

def get_final_dict(sentence, a1, a2, label):

  if pd.isna(a1["text"]) or pd.isna(a2["text"]):
    warnings.warn(f"Example dropped: NaN entity")
    return None

  idx1 = recompute_indices(sentence,a1["text"],a1["start"],a1["end"])
  idx2 = recompute_indices(sentence,a2["text"],a2["start"],a2["end"])

  if idx1[0]==-1 or idx2[0]==-1:
    warnings.warn(f"Example dropped: impossible to recompute the correct indices")
    return None

  return {'guid': id, 'sentence': sentence,
          'subject_entity': {'word': a1["text"],
                            'start_idx': idx1[0],
                            'end_idx': idx1[1],
                            'type': a1["type"]},
          'object_entity': {'word': a2["text"],
                            'start_idx': idx2[0],
                            'end_idx': idx2[1],
                            'type': a2["type"]},
          'label': label,
          'source': 'BioRED'}

def format_inputs(df_rel, df_ann, df_corpus, label_mapping):
    formatted_inputs = []
    drop_count = 0

    for i in tqdm(range(df_rel.shape[0])):

      rel = df_rel.iloc[i]
      label = rel["relation"]
      id = rel["pmid"]

      if len(df_corpus.loc[df_corpus["PMID"]==str(id)]["Testo"])==0:
        warnings.warn(f"Example dropped: relation points to non-existing document (pmid: {str(id)})")
        drop_count+=1
        continue
      sentence = df_corpus.loc[df_corpus["PMID"]==str(id)]["Testo"].iloc[0]

      #Entities Info
      arg1_id = rel["arg1_id"].replace("Arg1:", "")
      arg2_id = rel["arg2_id"].replace("Arg2:", "")
      ann1 = df_ann.loc[(df_ann['pmid'] == rel["pmid"]) & (df_ann['entity_id'] == arg1_id)]
      ann2 = df_ann.loc[(df_ann['pmid'] == rel["pmid"]) & (df_ann['entity_id'] == arg2_id)]

      if ann1.shape[0]==0:

        drop_count +=1
        continue
      elif ann2.shape[0]==0:
        drop_count +=1
        continue

      success = False
      if ann1.shape[0]>ann2.shape[0]:
        for j in range(ann1.shape[0]):
          a1 = ann1.iloc[j]
          diff = ann2['start']-a1['end']
          valid_idx = np.where(diff > 0)[0]
          if len(valid_idx)==0: continue
          idx = valid_idx[diff.iloc[valid_idx].argmin()]
          a2 = ann2.iloc[idx]
          formatted_input = get_final_dict(sentence,a1,a2,label_mapping[label])
          if formatted_input is not None:
            formatted_inputs.append(formatted_input)
            success = True
      else:
        for j in range(ann2.shape[0]):
          a2 = ann2.iloc[j]
          diff = ann1['end']-a2['start']
          valid_idx = np.where(diff < 0)[0]
          if len(valid_idx)==0: continue
          idx = valid_idx[diff.iloc[valid_idx].argmax()]
          a1 = ann1.iloc[idx]
          formatted_input = get_final_dict(sentence,a1,a2,label_mapping[label])
          if formatted_input is not None:
            formatted_inputs.append(formatted_input)
            success = True

      if not success:
        drop_count +=1
        continue
    print(f"\ndropped: {round(100*drop_count/i,2)}%")
    return(formatted_inputs)

In [ ]:
formatted_datasets = {
    "train":[],
    "test":[],
    "dev":[]
}

colnames = {"rel":["pmid","relation","arg1_id","arg2_id","novel"],
            "ann":["pmid","start","end","text","type","entity_id"]}


df_rel = pd.read_csv("train_REL.txt", sep='\t',names=colnames["rel"], header=None)

labels = np.unique(df_rel[["relation"]])
num_to_labels = dict(zip([i for i in range(len(labels))],labels))
labels_to_num = {v: k for k, v in num_to_labels.items()}

for key in formatted_datasets.keys():
  df_rel = pd.read_csv(key+"_REL.txt", sep='\t',names=colnames["rel"], header=None)
  df_ann = pd.read_csv(key+"_ANN.txt", sep='\t', names=colnames["ann"], header=None)
  f = open(key+"_CORPUS.json", encoding='utf-8')
  df_corpus = pd.DataFrame(json.load(f)["data"])

  f.close()

  formatted_datasets[key] = format_inputs(df_rel, df_ann, df_corpus, labels_to_num)

  0%|          | 0/4178 [00:00<?, ?it/s]<ipython-input-12-4141cbb4d0d0>:14: UserWarning: Example dropped: impossible to recompute the correct indices
  warnings.warn(f"Example dropped: impossible to recompute the correct indices")
100%|██████████| 4178/4178 [00:32<00:00, 128.80it/s]



dropped: 12.21%


  0%|          | 0/1163 [00:00<?, ?it/s]<ipython-input-12-4141cbb4d0d0>:14: UserWarning: Example dropped: impossible to recompute the correct indices
  warnings.warn(f"Example dropped: impossible to recompute the correct indices")
100%|██████████| 1163/1163 [00:07<00:00, 159.33it/s]



dropped: 21.26%


  0%|          | 0/1162 [00:00<?, ?it/s]<ipython-input-12-4141cbb4d0d0>:14: UserWarning: Example dropped: impossible to recompute the correct indices
  warnings.warn(f"Example dropped: impossible to recompute the correct indices")
100%|██████████| 1162/1162 [00:06<00:00, 170.13it/s]


dropped: 8.27%


In [ ]:
df_ann

,pmid,start,end,text,type,entity_id
0,14510914,0,25,Congenital hypothyroidism,DiseaseOrPhenotypicFeature,D003409
1,14510914,55,78,sodium/iodide symporter,GeneOrGeneProduct,6528
2,14510914,99,122,Iodide transport defect,DiseaseOrPhenotypicFeature,C564766
3,14510914,124,127,ITD,DiseaseOrPhenotypicFeature,C564766
4,14510914,168,192,inability of the thyroid,DiseaseOrPhenotypicFeature,D050033
...,...,...,...,...,...,...
3528,25119790,1552,1559,maleate,ChemicalEntity,C030272
3529,25119790,1568,1579,nephropathy,DiseaseOrPhenotypicFeature,D007674
3530,25119790,1716,1722,oxygen,ChemicalEntity,D010100
3531,25119790,1751,1772,respiratory complex I,ChemicalEntity,D042967


In [ ]:
df_corpus

,PMID,Testo
0,14510914,OBJECTIVE: Iodide transport defect (ITD) is a ...
1,15096016,A resurgence of interest in the surgical treat...
2,16152606,Nijmegen breakage syndrome (NBS) is a human au...
3,16277682,Mutations of the p62/Sequestosome 1 gene (p62/...
4,1671881,Classical phenylketonuria is an autosomal rece...
...,...,...
95,20621845,PURPOSE: This study was designed to establish ...
96,23872883,Cardiovascular diseases (CVDs) are the major h...
97,24333387,In the present study the role of glial activat...
98,24438483,BACKGROUND: Pre-treatment with lipid emulsions...


In [ ]:
df_rel

,pmid,relation,arg1_id,arg2_id,novel
0,14510914,Positive_Correlation,c|DEL|1314_1328|,D003409,Novel
1,14510914,Association,D050033,D007454,No
2,14510914,Positive_Correlation,p|DEL|439_443|,C564766,Novel
3,14510914,Positive_Correlation,p|DEL|439_443|,D003409,Novel
4,14510914,Association,C564766,D007454,No
...,...,...,...,...,...
1157,25119790,Association,C030272,D012964,Novel
1158,25119790,Association,C030272,D005947,Novel
1159,25119790,Positive_Correlation,C030272,D007674,No
1160,25119790,Negative_Correlation,D003474,D007674,Novel


In [ ]:
label_count = {}
for data in formatted_datasets["train"]:
    label = str(data['label'])+") "+num_to_labels[data['label']]
    if label not in label_count:
        label_count[label] = 1
    else:
        label_count[label] += 1

label_count = dict(sorted(label_count.items(), key=lambda x: x[0]))
label_count

{'0) Association': 9080,
 '1) Bind': 252,
 '2) Comparison': 178,
 '3) Conversion': 5,
 '4) Cotreatment': 166,
 '5) Drug_Interaction': 64,
 '6) Negative_Correlation': 3420,
 '7) Positive_Correlation': 4757}

In [ ]:
def add_entity_tokens(sentence, object_entity, subject_entity):
    obj_start_idx, obj_end_idx = object_entity['start_idx'], object_entity['end_idx']
    subj_start_idx, subj_end_idx = subject_entity['start_idx'], subject_entity['end_idx']

    if obj_start_idx < subj_start_idx:
        new_sentence = sentence[:obj_start_idx] + '<obj>' + sentence[obj_start_idx:obj_end_idx+1] + '</obj>' + \
                       sentence[obj_end_idx+1:subj_start_idx] + '<subj>' + sentence[subj_start_idx:subj_end_idx+1] + \
                       '</subj>' + sentence[subj_end_idx+1:]
    else:
        new_sentence = sentence[:subj_start_idx] + '<subj>' + sentence[subj_start_idx:subj_end_idx+1] + '</subj>' + \
                       sentence[subj_end_idx+1:obj_start_idx] + '<obj>' + sentence[obj_start_idx:obj_end_idx+1] + \
                       '</obj>' + sentence[obj_end_idx+1:]

    return new_sentence

def parse_re_dataset(dataset):
    sentences = []
    labels = []

    for data in dataset:
        sentence = add_entity_tokens(data['sentence'], data['object_entity'], data['subject_entity'])
        sentences.append(sentence)
        labels.append(data['label'])

    ds = Dataset.from_pandas(pd.DataFrame({'text': sentences,'label': labels}))

    return ds

In [ ]:
train_ds = parse_re_dataset(formatted_datasets["train"])
dev_ds = parse_re_dataset(formatted_datasets["dev"])
test_ds = parse_re_dataset(formatted_datasets["test"])

In [ ]:
train_ds[2]

{'text': 'The transcription factor hepatocyte nuclear factor (HNF)-6 is an upstream regulator of several genes involved in the pathogenesis of maturity-onset diabetes of the young. We therefore tested the hypothesis that variability in the HNF-6 gene is associated with subsets of Type II (non-insulin-dependent) diabetes mellitus and estimates of insulin secretion in glucose tolerant subjects.   We cloned the coding region as well as the intron-exon boundaries of the <subj>HNF-6</subj> gene. We then examined them on genomic DNA in six MODY probands without mutations in the MODY1, MODY3 and MODY4 genes and in 54 patients with late-onset <obj>Type II diabetes</obj> by combined single strand conformational polymorphism-heteroduplex analysis followed by direct sequencing of identified variants. An identified missense variant was examined in association studies and genotype-phenotype studies.   We identified two silent and one missense (Pro75 Ala) variant. In an association study the allelic

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

In [ ]:
seeds = [
         #3407,
         #6,
         11,
         61,
         1
        ]

batch_size = 16
learning_rate = 3e-5
epochs=7
weight_decay=0.01

In [ ]:
def prepare_tokenizer():
  tokenizer = AutoTokenizer.from_pretrained("IVN-RIN/medBIT-r3-plus")
  entity_special_tokens = {'additional_special_tokens': ['<obj>', '</obj>', '<subj>', '</subj>']}
  num_additional_special_tokens = tokenizer.add_special_tokens(entity_special_tokens)
  return tokenizer

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [ ]:
m_calc = {
    "precision":evaluate.load("precision"),
    "recall":evaluate.load("recall"),
    "accuracy":evaluate.load("accuracy"),
    "f1":evaluate.load("f1")
}
def compute_metrics(p):
  logits, labels = p
  predictions = np.argmax(logits, axis=1)
  return {'acc': m_calc["accuracy"].compute(references=labels, predictions=predictions)["accuracy"],
          'prec': m_calc["precision"].compute(references=labels, predictions=predictions, average="weighted")["precision"],
          'recall': m_calc["recall"].compute(references=labels, predictions=predictions, average="weighted")["recall"],
          'f1': m_calc["f1"].compute(references=labels, predictions=predictions, average="weighted")["f1"]}


In [ ]:
model_checkpoints=['a']

In [ ]:
!mkdir BioRED_ft_RE

mkdir: cannot create directory ‘BioRED_ft_RE’: File exists


In [ ]:
for model_checkpoint in model_checkpoints:
  results_collector = []
  for seed in seeds:
    set_seed(seed)

    tokenizer = prepare_tokenizer()
    model = AutoModelForSequenceClassification.from_pretrained("IVN-RIN/medBIT-r3-plus", num_labels=len(label_count))

    model.resize_token_embeddings(len(tokenizer))

    train_dataset = train_ds.map(preprocess_function, batched=True)
    test_dataset = test_ds.map(preprocess_function, batched=True)
    dev_dataset = dev_ds.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
        output_dir=f"/content/BioRED_ft_RE/{seed}",
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,
        load_best_model_at_end = True,
        metric_for_best_model = "f1",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        compute_metrics = compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
    )

    trainer.train()

    predictions,label_ids, metrics  = trainer.predict(test_dataset)
    print(metrics)
    metrics["seed"]=seed
    results_collector.append(metrics)

  df_results = pd.DataFrame(results_collector)
  display(df_results)
  df_results.to_csv(f'/content/RE_results_{os.path.basename(model_checkpoint)}.csv')
  files.download(f'/content/RE_results_{os.path.basename(model_checkpoint)}.csv')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/416 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/724k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

Map:   0%|          | 0/4647 [00:00<?, ? examples/s]

Map:   0%|          | 0/5223 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Acc,Prec,Recall,F1
1,0.826900,1.343713,0.552747,0.559754,0.552747,0.540823


KeyboardInterrupt: 

In [ ]:
from transformers import AutoAdapterModel

model = AutoAdapterModel.from_pretrained("microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext")
adapter_name = model.load_adapter("reginaboateng/umls_relational_extraction_adapter_PubMedBERT", source="hf", set_active=True)


In [ ]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")
model = AutoModel.from_pretrained("emilyalsentzer/Bio_ClinicalBERT")


In [ ]:
for model_checkpoint in model_checkpoints:
  results_collector = []
  for seed in seeds:
    set_seed(seed)

    tokenizer = prepare_tokenizer()
    model = AutoModelForSequenceClassification.from_pretrained("IVN-RIN/medBIT-r3-plus", num_labels=len(label_count))

    model.resize_token_embeddings(len(tokenizer))

    train_dataset = train_ds.map(preprocess_function, batched=True)
    test_dataset = test_ds.map(preprocess_function, batched=True)
    dev_dataset = dev_ds.map(preprocess_function, batched=True)

    training_args = TrainingArguments(
        output_dir=f"/content/BioRED_ft_RE/{seed}",
        evaluation_strategy="epoch",
        logging_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=3,
        load_best_model_at_end = True,
        metric_for_best_model = "f1",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=epochs,
        weight_decay=weight_decay
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=dev_dataset,
        tokenizer=tokenizer,
        compute_metrics = compute_metrics,
        callbacks = [EarlyStoppingCallback(early_stopping_patience = 3)]
    )

    trainer.train()

    predictions,label_ids, metrics  = trainer.predict(test_dataset)
    print(metrics)
    metrics["seed"]=seed
    results_collector.append(metrics)

  df_results = pd.DataFrame(results_collector)
  display(df_results)
  df_results.to_csv(f'/content/RE_results_{os.path.basename(model_checkpoint)}.csv')
  files.download(f'/content/RE_results_{os.path.basename(model_checkpoint)}.csv')

In [ ]:
class CustomBert(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        args = parser.parse_args()
        self.last_fc = nn.Linear((2 + args.add_word_num) * config.hidden_size, self.config.num_labels)

        self.init_weights()
        self.output_emebedding = None

    def forward(self, input_ids=None, attention_mask=None, token_type_ids=None, position_ids=None,
                head_mask=None, inputs_embeds=None, labels=None, e1_pos=None, e2_pos=None, aug_pos=None, sentence_data=None):

        if sentence_data is not None:
            device = torch.device("cuda")
            input_ids = sentence_data[0]#.to(device)
            attention_mask = sentence_data[1]#.to(device)
            labels = sentence_data[2]#.to(device)
            e1_pos = sentence_data[3]#.to(device)
            e2_pos = sentence_data[4]#.to(device)
            aug_pos = sentence_data[5]#.to(device)

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        e_pos_outputs = []
        sequence_output = outputs[0]
        for i in range(0, len(e1_pos)):
            obj_pos_output_i = sequence_output[i, obj_pos[i].item(), :]
            sub_pos_output_i = sequence_output[i, sub_pos[i].item(), :]
            e_pos_output_i = torch.cat((obj_pos_output_i, sub_pos_output_i), dim=0)

            for aug_index in aug_pos[i]:
                aug_pos_output_i = sequence_output[i, aug_index.item(), :]
                e_pos_output_i = torch.cat((e_pos_output_i, aug_pos_output_i), dim=0)
                pass
            e_pos_outputs.append(e_pos_output_i)
        e_pos_output = torch.stack(e_pos_outputs)


        self.output_emebedding = e_pos_output

        e_pos_output = self.dropout(e_pos_output)
        return e_pos_output

In [ ]:
import torch
import torch.nn as nn


class SimSiam(nn.Module):
    def __init__(self, base_encoder, dim=2048, pred_dim=512):
        super(SimSiam, self).__init__()
        self.encoder = base_encoder(num_classes=dim, zero_init_residual=True)

        prev_dim = self.encoder.fc.weight.shape[1]
        self.encoder.fc = nn.Sequential(nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # first layer
                                        nn.Linear(prev_dim, prev_dim, bias=False),
                                        nn.BatchNorm1d(prev_dim),
                                        nn.ReLU(inplace=True), # second layer
                                        self.encoder.fc,
                                        nn.BatchNorm1d(dim, affine=False)) # output layer
        self.encoder.fc[6].bias.requires_grad = False # hack: not use bias as it is followed by BN

        self.predictor = nn.Sequential(nn.Linear(dim, pred_dim, bias=False),
                                        nn.BatchNorm1d(pred_dim),
                                        nn.ReLU(inplace=True), # hidden layer
                                        nn.Linear(pred_dim, dim)) # output layer

    def forward(self, x1, x2):
        z1 = self.encoder(x1)
        z2 = self.encoder(x2)

        p1 = self.predictor(z1)
        p2 = self.predictor(z2)

        return p1, p2, z1.detach(), z2.detach()

In [ ]:
from PIL import ImageFilter
import random


class TwoCropsTransform:

    def __init__(self, base_transform):
        self.base_transform = base_transform

    def __call__(self, x):
        q = self.base_transform(x)
        k = self.base_transform(x)
        return [q, k]


class GaussianBlur(object):

    def __init__(self, sigma=[.1, 2.]):
        self.sigma = sigma

    def __call__(self, x):
        sigma = random.uniform(self.sigma[0], self.sigma[1])
        x = x.filter(ImageFilter.GaussianBlur(radius=sigma))
        return x